In [1]:
import sys
#This line imports the 'sys' module, which provides access to some variables used 
# or maintained by the interpreter and to functions that interact strongly with the interpreter. It is not used in the rest of the code.

import spotipy 
#Python library for the Spotify Web API. 
# It allows developers to interact with Spotify's music 
#streaming services and data about artists, albums, and songs.
from spotipy.oauth2 import SpotifyClientCredentials
#'SpotifyClientCredentials' class from the 'spotipy.oauth2' module. 
# This class is used to authenticate and authorize access to the Spotify Web API 
# using client credentials.

import pandas as pd
#This line imports the 'pandas' module and renames it as 'pd'. 
# Pandas is a popular data manipulation and analysis library for Python.

import os
#This line imports the 'os' module, which provides a way of interacting 
# with the operating system in a portable way. 
# It allows Python code to access environment variables, files, directories, 
# and other operating system resources.

In [2]:
client_id = os.getenv("client_id")
client_secret = os.getenv("client_secret")

In [3]:
credentials = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

In [4]:
sp = spotipy.Spotify(client_credentials_manager=credentials)

In [6]:
def get_playlist_info(link):

    # Get playlist URI from link
    playlist_URI = link.split("/")[-1].split("?")[0]

    # Get all tracks from playlist I kept 3 because I know I have no more than 300 tracks
    offset = 0
    all_data = []
    for i in range(3):
        all_data.append(sp.playlist_tracks(playlist_URI, offset=offset)["items"])
        offset += 100

    # Store track information in a dictionary
    basic_info = {"song": [], 
              "artist_1": [], 
              "artist_id_1": [], 
              "album": [], 
              "album_id": [],
              "duration": [],
              "date": [],
              "date_album": [], 
              "imagen": [], 
              "url_song": [], 
              "total_tracks": []}

    for element in range(len(all_data)):
        for song in range(len(all_data[element])):
            basic_info["song"].append(all_data[element][song]["track"]["name"])
            basic_info["album"].append(all_data[element][song]["track"]["album"]["name"])
            basic_info["album_id"].append(all_data[element][song]["track"]["album"]['id'])
            basic_info["duration"].append(all_data[element][song]["track"]["duration_ms"])
            basic_info["date"].append(all_data[element][song]["added_at"])
            basic_info["date_album"].append(all_data[element][song]["track"]["album"]["release_date"])
            basic_info["imagen"].append(all_data[element][song]["track"]["album"]["images"][0]["url"])
            basic_info["url_song"].append(all_data[element][song]["track"]["external_urls"]["spotify"])
            basic_info["total_tracks"].append(all_data[element][song]["track"]["album"]["total_tracks"])
        
            num_artists = len(all_data[element][song]["track"]["artists"])
            for i in range(num_artists):
                artist_key = f"artist_{i+1}"
                artist_id_key = f"artist_id_{i+1}"
                basic_info.setdefault(artist_key, [])
                basic_info.setdefault(artist_id_key, [])
                basic_info[artist_key].append(all_data[element][song]["track"]["artists"][i]["name"])
                basic_info[artist_id_key].append(all_data[element][song]["track"]["artists"][i]["id"])
        
        # Add "none" to extra artist and artist_id columns if there is no value
            for j in range(num_artists+1, 6):
                artist_key = f"artist_{j}"
                artist_id_key = f"artist_id_{j}"
                basic_info.setdefault(artist_key, [])
                basic_info.setdefault(artist_id_key, [])
                basic_info[artist_key].append("none")
                basic_info[artist_id_key].append("none")

    # Create DataFrame from dictionary
        df = pd.DataFrame(basic_info)

    return df


In [9]:
link = "https://open.spotify.com/playlist/2ZZQAzWW9nHlLh8QGWan3l?si=65eaa0f24dab4725"
playlist_80 = get_playlist_info(link)
playlist_80.sample(5)

,song,artist_1,artist_id_1,album,album_id,duration,date,date_album,imagen,url_song,total_tracks,artist_2,artist_id_2,artist_3,artist_id_3,artist_4,artist_id_4,artist_5,artist_id_5
85,Wish You Were Here - 2016 Remaster,Fleetwood Mac,08GQAI4eElDnROBrJRGE0X,Mirage (2016 Remaster),2962eYkQa0R7EnIGZqeARS,291092,2022-10-01T07:01:14Z,1982-06-18,https://i.scdn.co/image/ab67616d0000b273b1751a...,https://open.spotify.com/track/1MagANszakeiDxz...,12,none,none,none,none,none,none,none,none
57,Every Little Thing She Does Is Magic,The Police,5NGO30tJxFlKixkPSgXcFE,Ghost In The Machine (Remastered 2003),5jkwdY6jS1Hzi8epr6HW7h,260573,2022-09-28T06:45:56Z,1981-10-02,https://i.scdn.co/image/ab67616d0000b273b78799...,https://open.spotify.com/track/44aTAUBF0g6sMkM...,11,none,none,none,none,none,none,none,none
236,Let Me Go Home,The Alan Parsons Project,2m62cc253Xvd9qYQ8d2X3d,Ammonia Avenue (Expanded Edition),0RuvKbx3Zwa1LDkL9uEuTX,201386,2023-04-12T15:55:48Z,1984-02-07,https://i.scdn.co/image/ab67616d0000b2738a6678...,https://open.spotify.com/track/5fowo9X85P352px...,17,none,none,none,none,none,none,none,none
132,Seven Seas Of Rhye - Remastered 2011,Queen,1dfeR4HaWDbWqFHLkxsg1d,Greatest Hits (Remastered),3VWrUk4vBznMYXGMPc7dRB,167026,2022-12-28T09:53:33Z,1981-10-26,https://i.scdn.co/image/ab67616d0000b273bb19d0...,https://open.spotify.com/track/5u99eyhU43QoLeX...,17,none,none,none,none,none,none,none,none
179,More Than This,Roxy Music,3fhOTtm0LBJ3Ojn4hIljLo,Avalon,3JXODSjT9mUz2lIb4YIErw,270651,2023-02-04T16:10:52Z,1982-05-01,https://i.scdn.co/image/ab67616d0000b273c79727...,https://open.spotify.com/track/6N7gPTru90HYLRU...,10,none,none,none,none,none,none,none,none


In [10]:
playlist_80.to_csv("1980_LPs.csv", index=False)

In [12]:
link = "https://open.spotify.com/playlist/2Lq1zuWTgPBmaCXLw1kMeL?si=c624539415174b4c"
playlist_90 = get_playlist_info(link)
playlist_90.sample(5)

,song,artist_1,artist_id_1,album,album_id,duration,date,date_album,imagen,url_song,total_tracks,artist_2,artist_id_2,artist_3,artist_id_3,artist_4,artist_id_4,artist_5,artist_id_5
10,Feel Like Going Home,The Notting Hillbillies,1dqalNipWI6wAnatpwQtWu,Missing... Presumed Having A Good Time,4HyXgM6clwGCwHe2Yhxt1B,292773,2023-03-09T08:29:09Z,1990-01-01,https://i.scdn.co/image/ab67616d0000b2736d6688...,https://open.spotify.com/track/0qMPttvKZkUXyDg...,11,none,none,none,none,none,none,none,none
36,Clear Light,Mike Oldfield,562Od3CffWedyz2BbeYWVn,Tubular Bells II,0Z9Uio2xZCDztGCuozTPaH,348026,2023-03-09T08:31:39Z,1992,https://i.scdn.co/image/ab67616d0000b27397c14c...,https://open.spotify.com/track/4lToqtluiO2UEdb...,14,none,none,none,none,none,none,none,none
30,Sweet Dreams - Live,Emmylou Harris,5s6TJEuHTr9GR894wc6VfP,Ramble in Music City: The Lost Concert (Live),67Coweh6bxMgu7wijLpzGa,234900,2023-03-09T08:30:43Z,2021-09-03,https://i.scdn.co/image/ab67616d0000b27381b8cd...,https://open.spotify.com/track/6veBDZN8RJoyQH9...,23,The Nash Ramblers,6kfIzjkF8DgvTSw0XlAJOh,none,none,none,none,none,none
39,Red Dawn,Mike Oldfield,562Od3CffWedyz2BbeYWVn,Tubular Bells II,0Z9Uio2xZCDztGCuozTPaH,110000,2023-03-09T08:31:39Z,1992,https://i.scdn.co/image/ab67616d0000b27397c14c...,https://open.spotify.com/track/0Erh1lZACu0YsGn...,14,none,none,none,none,none,none,none,none
28,Born to Run - Live,Emmylou Harris,5s6TJEuHTr9GR894wc6VfP,Ramble in Music City: The Lost Concert (Live),67Coweh6bxMgu7wijLpzGa,227207,2023-03-09T08:30:43Z,2021-09-03,https://i.scdn.co/image/ab67616d0000b27381b8cd...,https://open.spotify.com/track/2rxrZeumJEiloca...,23,The Nash Ramblers,6kfIzjkF8DgvTSw0XlAJOh,none,none,none,none,none,none


In [13]:
playlist_90.to_csv("1990_LPs.csv", index=False)